- SVL 예측을 위한 선형 모델링
- 개인적으로 **추천하지 않는** 방식이지만, 예측 보다는 benchmark가 필요하다는 목적 하에 진행
- 비선형 모델이 훨씬 적절하겠지만, 유지보수 관점에서 (엑셀) ridge 모델로 parameter 추출

In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
import pandas_profiling

import seaborn as sns

In [6]:
df = pd.read_excel('svl_table.xlsx')
df = df[['SVL30', 'workinghour', 'Call', 'NewcomerP', 'ATT']]
df['call/hour'] = df.Call/df.workinghour
df.tail()

SVL30  workinghour  Call  NewcomerP    ATT  call/hour
82  0.8545         3898  6658   0.067537  249.0   1.708055
83  0.8610         3309  5143   0.067537  260.0   1.554246
84  0.8271         3927  6126   0.067537  251.0   1.559969
85  0.8398         3798  6435   0.082132  257.0   1.694313
86  0.8303         4258  7620   0.082132  255.0   1.789573

In [26]:
df.profile_report()

In [7]:
y = df.SVL30
X = df.iloc[:,1:]
X.tail()

workinghour  Call  NewcomerP    ATT  call/hour
82         3898  6658   0.067537  249.0   1.708055
83         3309  5143   0.067537  260.0   1.554246
84         3927  6126   0.067537  251.0   1.559969
85         3798  6435   0.082132  257.0   1.694313
86         4258  7620   0.082132  255.0   1.789573

In [47]:
X = X.drop('call/hour', axis=1)
X.tail()

workinghour  Call  NewcomerP    ATT
82         3898  6658   0.067537  249.0
83         3309  5143   0.067537  260.0
84         3927  6126   0.067537  251.0
85         3798  6435   0.082132  257.0
86         4258  7620   0.082132  255.0

## OLS

In [51]:
ols_model = sm.OLS(y, sm.add_constant(X))
ols_model = ols_model.fit()

In [52]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  SVL30   R-squared:                       0.477
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     18.69
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           5.94e-11
Time:                        15:09:46   Log-Likelihood:                 215.13
No. Observations:                  87   AIC:                            -420.3
Df Residuals:                      82   BIC:                            -407.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.3113      0.081     16.262      0.000       1.151       1.472
workinghour   2.01e-05   6.37e-06      3.155      0.002    7.42e-06    3.28e-05
Call        -2.345e-05   2.88e-06     -8.146      0.000   -2.92e-05   -1.77e-05
NewcomerP      -0.2109      0.046     -4.618      0.000      -0.302      -0.120
ATT            -0.0014      0.000     -4.957      0.000      -0.002      -0.001
==============================================================================
Omnibus:                        3.146   Durbin-Watson:                   1.019
Prob(Omnibus):                  0.207   Jarque-Bera (JB):                2.481
Skew:                          -0.291   Prob(JB):                        0.289
Kurtosis:                       3.588   Cond. No.                     2.57e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.57e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
mean_absolute_error(ols_model.predict(sm.add_constant(X)), y)

c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


0.015397342824670675

In [37]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [57]:
import numpy as np

In [59]:
for alpha in [0.01, 0.5, 0.3, 0.1, 1]:
    ridge = Ridge(alpha=alpha)
    ridge.fit(sm.add_constant(X), y)
    print('alpha = {}, MAE = {}'.format(alpha, mean_absolute_error(ridge.predict(sm.add_constant(X)), y)))


alpha = 0.01, MAE = 0.015404144263026191
alpha = 0.5, MAE = 0.01706620362880131
alpha = 0.3, MAE = 0.016568968896105846
alpha = 0.1, MAE = 0.015754385879714984
alpha = 1, MAE = 0.017608419130884272


c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)
c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out

## train_test

In [121]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.1, random_state = 1)

- with ols

In [122]:
ols_model = sm.OLS(train_y, sm.add_constant(train_x))
ols_model = ols_model.fit()
ols_model.summary()

c:\users\chpar10\appdata\local\programs\python\python37\lib\site-packages\numpy\core\fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  SVL30   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     13.28
Date:                Wed, 11 Mar 2020   Prob (F-statistic):           3.54e-08
Time:                        15:57:01   Log-Likelihood:                 194.57
No. Observations:                  78   AIC:                            -379.1
Df Residuals:                      73   BIC:                            -367.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           1.2780      0.085     15.049      0.000       1.109       1.447
workinghour  2.154e-05   6.45e-06      3.341      0.001    8.69e-06    3.44e-05
Call        -2.198e-05   3.13e-06     -7.027      0.000   -2.82e-05   -1.57e-05
NewcomerP      -0.2060      0.048     -4.297      0.000      -0.302      -0.110
ATT            -0.0013      0.000     -4.423      0.000      -0.002      -0.001
==============================================================================
Omnibus:                        6.065   Durbin-Watson:                   1.639
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                5.536
Skew:                          -0.494   Prob(JB):                       0.0628
Kurtosis:                       3.852   Cond. No.                     2.59e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.59e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [123]:
mean_absolute_error(ols_model.predict(sm.add_constant(test_x)), test_y)

0.021543240076240717

In [124]:
ridge = Ridge(alpha=alpha)
ridge.fit(train_x, train_y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [125]:
mean_absolute_error(ridge.predict(test_x), test_y)

0.02396054458945645

In [126]:
tmp_index = ['intercept']
tmp_index.extend([i for i in X.columns])
tmp_val = [ridge.intercept_]
tmp_val.extend(ridge.coef_)

pd.DataFrame(tmp_val, index = tmp_index)

0
intercept    1.047667
workinghour  0.000019
Call        -0.000017
NewcomerP   -0.032217
ATT         -0.000490